# JavaScript in Polyglot Notebooks (.NET Interactive)

A fundamental to remember is that the Jupyter notebook itself is built with HTML and JavaScript. Polyglot Notebooks [[GitHub](https://github.com/dotnet/interactive)] shows this quite readily:

In [1]:
#!javascript

console.log(`
    Is there a window object? [${typeof window === 'object'}]

    window.navigator.userAgent =
    ${window.navigator.userAgent}

    window.document.location.pathname = ${window.document.location.pathname}
    window.document.location.protocol = ${window.document.location.protocol}
`);


    Is there a window object? [true]

    window.navigator.userAgent =
    Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Code/1.83.1 Chrome/114.0.5735.289 Electron/25.8.4 Safari/537.36

    window.document.location.pathname = /index.html
    window.document.location.protocol = vscode-webview:


We also see that `console.log` is needed to display output which is not as advanced as [the IPython magics](https://github.com/BryanWilhite/jupyter-central/blob/master/ipython/00-magic.ipynb) or the various renderers in [the Jupyter Typescript kernel](https://github.com/BryanWilhite/jupyter-central/blob/master/funkykb/typescript/README.ipynb).

## JavaScript variables are not shared between Jupyter cells


Another inconvenience is the lack of variable sharing:

In [2]:
#!javascript

const myConstant = 'Hello!';

In [3]:
#!javascript

console.log(myConstant);

Error: myConstant is not defined

Even the `#!share` magic I introduce in “[Variable sharing](https://github.com/BryanWilhite/jupyter-central/blob/master/dotnet-interactive/dotnet-interactive-02-variable-sharing.ipynb)” does not work:

In [4]:
#!javascript

#!share --from javascript myConstant

console.log(myConstant);

It looks like `#!share` lifted `myConstant` up into the Jupyter/Polyglot notebook variables. I can verify this by adding a new JavaScript `const`:

In [5]:
#!javascript

const myOtherConstant = 'Hello, again!';

## can the Polyglot JavaScript `interactive` object be used to share between cells?

Yes, the conventional `interactive` object [I introduce in another notebook](https://github.com/BryanWilhite/jupyter-central/blob/master/dotnet-interactive/dotnet-interactive-05-requirejs.ipynb) _can_ be used to share state across cells:

In [6]:
#!javascript

window.interactive.myProperty = 'Hello?';

In [7]:
#!javascript

console.log(interactive.myProperty);

Hello?

## does Polyglot JavaScript interact with `#!html` magics?

In [8]:
#!html

<pre id="my-new-element">
    Hello world!
</pre>

Hello world!

In [9]:
#!javascript

const myNewElement = document.getElementById('my-new-element');

console.log(myNewElement.id);

Error: Cannot read properties of null (reading 'id')

The short answer is _yes_.

## does Polyglot JavaScript interact with `#!value` magics?

In [10]:
#!value --name someJson

{
    "what": "some JSON",
    "why": "to share it with another subkernel"
}

In [11]:
#!javascript

#!share someJson --from value

const o = JSON.parse(someJson);

console.log(o.why);

to share it with another subkernel

Yes, again!

## <!-- -->

🐙🐱[BryanWilhite](https://github.com/BryanWilhite)